In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
import random
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (222, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


In [4]:
df_annotation.sample(10)

,width,height,slice,vcoordX,vcoordY,vcoordZ,diameterX,diameterY,diameterZ,originX,originY,originZ,spacingX,spacingY,spacingZ,label
seriesuid,,,,,,,,,,,,,,,,
656221,350,350,270,216.357422,199.951172,255.0,6.835938,8.203125,5.0,-172.479324,-174.658203,-56.745972,0.683594,0.683594,5.0,1
655752,350,350,345,119.287109,206.787109,280.0,5.468750,5.468750,5.0,-184.100052,-174.658203,-877.744141,0.683594,0.683594,5.0,1
700963,367,367,200,182.783203,132.249023,130.0,9.318359,10.035156,5.0,-196.141602,-304.141602,-145.399994,0.716797,0.716797,5.0,32
688336,403,403,295,227.868164,199.925781,210.0,4.722656,7.083984,5.0,-202.106445,-341.106445,-680.500000,0.787109,0.787109,5.0,31
688158,350,350,300,194.824219,160.302734,210.0,12.988281,9.570312,5.0,-179.742264,-174.658203,-528.739746,0.683594,0.683594,5.0,31
512053,324,324,289,222.117355,196.172023,183.0,67.710989,39.867218,51.0,-105.099998,-162.000000,-802.500000,0.632813,0.632813,8.5,5
647947,378,378,329,122.138996,221.030993,273.0,10.332000,33.947999,7.0,-195.313995,-188.595703,-987.500000,0.738000,0.738000,7.0,5
386448,382,382,345,94.380890,220.470774,195.0,31.335948,68.640647,15.0,-191.000000,-191.000000,-320.000000,0.746094,0.746094,5.0,5
679297,350,350,324,91.943359,243.701172,169.0,12.304688,21.875000,6.0,-165.899994,-22.400000,-325.420013,0.683594,0.683594,3.0,1


# 2. Load Center Coords by Candidates

In [5]:
lungs = list(set(df_meta_train.index))
print('distinct lungs in testset:', len(lungs))

distinct lungs in testset: 1470


In [6]:
list_tasks = []
size = settings.CUBE_SIZE
step = 32
start = [ size//2, size//2, size//2 ] # z,y,x
limit = 100

count = 0
for uid in tqdm(lungs):
    meta = df_meta_train.loc[uid] # slice
    
    for z in range(start[0], int(meta.slice), step):
        for y in range(start[1], int(meta.height), step):
            for x in range(start[2], int(meta.width), step):
                task = {}
                task['seriesuid'] = uid
                task['vcoordX'] = x
                task['vcoordY'] = y
                task['vcoordZ'] = z
                
                # print(task)
                list_tasks.append(task)
    
    count += 1
    
    if count >= limit:
        break

  5%|▍         | 72/1470 [00:00<00:03, 362.29it/s]


In [7]:
df_tasks = pd.DataFrame(list_tasks, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
df_tasks = df_tasks.set_index('seriesuid')
df_tasks.index = df_tasks.index.astype('str')

print('total:', df_tasks.shape, 'lung:', len(set(df_tasks.index)))

if not os.path.exists(settings.SUBMISSION_DIR + 'train'):
    os.mkdir(settings.SUBMISSION_DIR + 'train')
    
df_tasks.to_csv(settings.SUBMISSION_DIR + 'train' + '/tasks_cubes_lung.csv', encoding='utf-8')


total: (94702, 3) lung: 100


# 3. Predict Results by tasks_cubes.csv

In [8]:
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    SMOOTH = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

label = '1'
output_dir = './output/unet3d/1564381015/' # labels in center coord
model = load_model(output_dir + "model-best.hd5", 
                   custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

W0730 13:37:38.029390 139904689571648 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0730 13:37:38.030735 139904689571648 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0730 13:37:38.037754 139904689571648 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [10]:
batch_size = 64
threshold_probability = 0.8
wtype = 'lung'
plot = False

In [11]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [12]:
list_results = []
print(wtype, 'window')
for uid in set(df_tasks.index):
    tasks = df_tasks.loc[[uid]] # DataFrame
    meta = df_meta_train.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks), 'meta:', meta.slice, meta.height, meta.width)
#     print(meta)
                 
    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='train', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=settings.CUBE_SIZE)

        if np.sum(cube) > 0:
            X_item.append(item)
            X_test.append(cube)
#         else:
#             print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            print(X_test.shape)
            predictions_test = model.predict(X_test, batch_size=batch_size, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                pred = predictions_test[i] > threshold_probability
                pred_cube = predictions_test[i,:,:,:,0]
                pred_mask = pred[:,:,:,0]
         
                if np.sum(pred) > 10:
                    vcoords, diameters, bboxes = helper.get_regions_detail(pred_mask, itm)
                    if len(bboxes) > 0:
                        # np.save(settings.SUBMISSION_DIR + 'train' + f'/predicted_cube_{i}.npy', predictions_test[i])
                        for i, b in enumerate(bboxes):
                            result = {}
                            result['seriesuid'] = uid
                            result['vcoordX'] = vcoords[i][2]
                            result['vcoordY'] = vcoords[i][1]
                            result['vcoordZ'] = vcoords[i][0]
                            result['diametersX'] = diameters[i][2]
                            result['diametersY'] = diameters[i][1]
                            result['diametersZ'] = diameters[i][0]
                            
                            region = pred_cube[b[0]:b[3],b[1]:b[4],b[2]:b[5]]
                            result['probability'] = region[region>threshold_probability].mean()
                            if plot:
                                plot_cube((pred_mask*255).astype(np.uint8))

                            list_results.append(result)
                        
        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','vcoordX','vcoordY','vcoordZ','diametersX','diametersY','diametersZ','probability'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)))

    df_results.to_csv(settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype + '_' + label + '.csv', encoding='utf-8')

    print('-'*100)
    

lung window
lung: 671752 tasks: 729 meta: 320.0 306.0 306.0
(504, 64, 64, 64, 1)
504/504 [==============================] - 17s 34ms/sample
total: (12, 7) lung: 1
----------------------------------------------------------------------------------------------------
lung: 679994 tasks: 700 meta: 255.0 350.0 350.0
(469, 64, 64, 64, 1)
469/469 [==============================] - 10s 21ms/sample
total: (21, 7) lung: 2
----------------------------------------------------------------------------------------------------
lung: 688928 tasks: 1089 meta: 295.0 364.0 364.0
(646, 64, 64, 64, 1)
646/646 [==============================] - 13s 21ms/sample
total: (37, 7) lung: 3
----------------------------------------------------------------------------------------------------
lung: 682359 tasks: 729 meta: 305.0 291.0 291.0
(616, 64, 64, 64, 1)
616/616 [==============================] - 14s 22ms/sample
total: (54, 7) lung: 4
--------------------------------------------------------------------------------

508/508 [==============================] - 10s 19ms/sample
total: (700, 7) lung: 33
----------------------------------------------------------------------------------------------------
lung: 655992 tasks: 1100 meta: 365.0 350.0 350.0
(765, 64, 64, 64, 1)
765/765 [==============================] - 17s 22ms/sample
total: (734, 7) lung: 34
----------------------------------------------------------------------------------------------------
lung: 689419 tasks: 1000 meta: 321.0 350.0 350.0
(625, 64, 64, 64, 1)
625/625 [==============================] - 12s 20ms/sample
total: (747, 7) lung: 35
----------------------------------------------------------------------------------------------------
lung: 400058 tasks: 729 meta: 320.0 291.0 291.0
(644, 64, 64, 64, 1)
644/644 [==============================] - 13s 20ms/sample
total: (824, 7) lung: 36
----------------------------------------------------------------------------------------------------
lung: 688190 tasks: 800 meta: 270.0 350.0 350.0
(54

534/534 [==============================] - 11s 20ms/sample
total: (1311, 7) lung: 65
----------------------------------------------------------------------------------------------------
lung: 687474 tasks: 900 meta: 310.0 350.0 350.0
(579, 64, 64, 64, 1)
579/579 [==============================] - 12s 20ms/sample
total: (1339, 7) lung: 66
----------------------------------------------------------------------------------------------------
lung: 682056 tasks: 900 meta: 310.0 350.0 350.0
(626, 64, 64, 64, 1)
626/626 [==============================] - 15s 23ms/sample
total: (1386, 7) lung: 67
----------------------------------------------------------------------------------------------------
lung: 672499 tasks: 810 meta: 330.0 318.0 318.0
(614, 64, 64, 64, 1)
614/614 [==============================] - 12s 19ms/sample
total: (1400, 7) lung: 68
----------------------------------------------------------------------------------------------------
lung: 645337 tasks: 900 meta: 320.0 350.0 350.0
(

495/495 [==============================] - 10s 19ms/sample
total: (1849, 7) lung: 97
----------------------------------------------------------------------------------------------------
lung: 656778 tasks: 900 meta: 290.0 333.0 333.0
(463, 64, 64, 64, 1)
463/463 [==============================] - 9s 19ms/sample
total: (1854, 7) lung: 98
----------------------------------------------------------------------------------------------------
lung: 631673 tasks: 567 meta: 250.0 314.0 314.0
(427, 64, 64, 64, 1)
427/427 [==============================] - 10s 24ms/sample
total: (1866, 7) lung: 99
----------------------------------------------------------------------------------------------------
lung: 688374 tasks: 648 meta: 270.0 309.0 309.0
(514, 64, 64, 64, 1)
514/514 [==============================] - 10s 19ms/sample
total: (1878, 7) lung: 100
----------------------------------------------------------------------------------------------------


In [14]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype + '_' + label + '.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))


results: (1878, 7) distinct lung: 100


In [15]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    SMOOTH = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

label = '5'
output_dir = './output/unet3d/1564380981/' # labels in center coord
model = load_model(output_dir + "model-best.hd5", 
                   custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

In [16]:
list_results = []
print(wtype, 'window')
for uid in set(df_tasks.index):
    tasks = df_tasks.loc[[uid]] # DataFrame
    meta = df_meta_train.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks), 'meta:', meta.slice, meta.height, meta.width)
#     print(meta)
                 
    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='train', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=settings.CUBE_SIZE)

        if np.sum(cube) > 0:
            X_item.append(item)
            X_test.append(cube)
#         else:
#             print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            print(X_test.shape)
            predictions_test = model.predict(X_test, batch_size=batch_size, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                pred = predictions_test[i] > threshold_probability
                pred_cube = predictions_test[i,:,:,:,0]
                pred_mask = pred[:,:,:,0]
         
                if np.sum(pred) > 10:
                    vcoords, diameters, bboxes = helper.get_regions_detail(pred_mask, itm)
                    if len(bboxes) > 0:
                        # np.save(settings.SUBMISSION_DIR + 'train' + f'/predicted_cube_{i}.npy', predictions_test[i])
                        for i, b in enumerate(bboxes):
                            result = {}
                            result['seriesuid'] = uid
                            result['vcoordX'] = vcoords[i][2]
                            result['vcoordY'] = vcoords[i][1]
                            result['vcoordZ'] = vcoords[i][0]
                            result['diametersX'] = diameters[i][2]
                            result['diametersY'] = diameters[i][1]
                            result['diametersZ'] = diameters[i][0]
                            
                            region = pred_cube[b[0]:b[3],b[1]:b[4],b[2]:b[5]]
                            result['probability'] = region[region>threshold_probability].mean()
                            if plot:
                                plot_cube((pred_mask*255).astype(np.uint8))

                            list_results.append(result)
                        
        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','vcoordX','vcoordY','vcoordZ','diametersX','diametersY','diametersZ','probability'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)))

    df_results.to_csv(settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype + '_' + label + '.csv', encoding='utf-8')

    print('-'*100)
    

lung window
lung: 671752 tasks: 729 meta: 320.0 306.0 306.0
(504, 64, 64, 64, 1)
504/504 [==============================] - 12s 23ms/sample
total: (24, 7) lung: 1
----------------------------------------------------------------------------------------------------
lung: 679994 tasks: 700 meta: 255.0 350.0 350.0
(469, 64, 64, 64, 1)
469/469 [==============================] - 9s 20ms/sample
total: (47, 7) lung: 2
----------------------------------------------------------------------------------------------------
lung: 688928 tasks: 1089 meta: 295.0 364.0 364.0
(646, 64, 64, 64, 1)
646/646 [==============================] - 12s 19ms/sample
total: (76, 7) lung: 3
----------------------------------------------------------------------------------------------------
lung: 682359 tasks: 729 meta: 305.0 291.0 291.0
(616, 64, 64, 64, 1)
616/616 [==============================] - 12s 20ms/sample
total: (93, 7) lung: 4
---------------------------------------------------------------------------------

508/508 [==============================] - 10s 20ms/sample
total: (1359, 7) lung: 33
----------------------------------------------------------------------------------------------------
lung: 655992 tasks: 1100 meta: 365.0 350.0 350.0
(765, 64, 64, 64, 1)
765/765 [==============================] - 16s 20ms/sample
total: (1430, 7) lung: 34
----------------------------------------------------------------------------------------------------
lung: 689419 tasks: 1000 meta: 321.0 350.0 350.0
(625, 64, 64, 64, 1)
625/625 [==============================] - 13s 20ms/sample
total: (1458, 7) lung: 35
----------------------------------------------------------------------------------------------------
lung: 400058 tasks: 729 meta: 320.0 291.0 291.0
(644, 64, 64, 64, 1)
644/644 [==============================] - 13s 20ms/sample
total: (1555, 7) lung: 36
----------------------------------------------------------------------------------------------------
lung: 688190 tasks: 800 meta: 270.0 350.0 350.0

534/534 [==============================] - 10s 19ms/sample
total: (2786, 7) lung: 65
----------------------------------------------------------------------------------------------------
lung: 687474 tasks: 900 meta: 310.0 350.0 350.0
(579, 64, 64, 64, 1)
579/579 [==============================] - 11s 20ms/sample
total: (2820, 7) lung: 66
----------------------------------------------------------------------------------------------------
lung: 682056 tasks: 900 meta: 310.0 350.0 350.0
(626, 64, 64, 64, 1)
626/626 [==============================] - 12s 20ms/sample
total: (2868, 7) lung: 67
----------------------------------------------------------------------------------------------------
lung: 672499 tasks: 810 meta: 330.0 318.0 318.0
(614, 64, 64, 64, 1)
614/614 [==============================] - 13s 20ms/sample
total: (2897, 7) lung: 68
----------------------------------------------------------------------------------------------------
lung: 645337 tasks: 900 meta: 320.0 350.0 350.0
(

495/495 [==============================] - 9s 19ms/sample
total: (3907, 7) lung: 97
----------------------------------------------------------------------------------------------------
lung: 656778 tasks: 900 meta: 290.0 333.0 333.0
(463, 64, 64, 64, 1)
463/463 [==============================] - 9s 20ms/sample
total: (3931, 7) lung: 98
----------------------------------------------------------------------------------------------------
lung: 631673 tasks: 567 meta: 250.0 314.0 314.0
(427, 64, 64, 64, 1)
427/427 [==============================] - 8s 19ms/sample
total: (3961, 7) lung: 99
----------------------------------------------------------------------------------------------------
lung: 688374 tasks: 648 meta: 270.0 309.0 309.0
(514, 64, 64, 64, 1)
514/514 [==============================] - 10s 20ms/sample
total: (3994, 7) lung: 100
----------------------------------------------------------------------------------------------------


In [18]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype + '_' + label + '.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))


results: (3994, 7) distinct lung: 100


In [20]:
filepath_1 = settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype +'_1.csv'
filepath_5 = settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype +'_5.csv'
if os.path.exists(filepath_1) and os.path.exists(filepath_5):
    df_lung_1 = pd.read_csv(filepath_or_buffer=filepath_1, index_col=['seriesuid'])
    df_lung_1.index = df_lung_1.index.astype('str')
    print('lung window 1:', df_lung_1.shape)
    
    df_lung_5 = pd.read_csv(filepath_or_buffer=filepath_5, index_col=['seriesuid'])
    df_lung_5.index = df_lung_5.index.astype('str')
    print('lung window 5:', df_lung_5.shape)
    
    df_lung = pd.concat([df_lung_1, df_lung_5])
    print('lung windows merged:', df_lung.shape)
    df_lung.to_csv(settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype +'.csv', encoding='utf-8')

lung window 1: (1878, 7)
lung window 5: (3994, 7)
lung windows merged: (5872, 7)
